In [1]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

In [2]:
def transform_similarity_matrix(matrix, midpoint=0.5, sharpness=10, scale=10):
    # Normalize to [-1, 1]
    matrix = 2 * (matrix - matrix.min()) / (matrix.max() - matrix.min()) - 1
    # Apply sigmoid transformation
    def sigmoid_reward(x):
        return scale * (1 / (1 + np.exp(-sharpness * (x - midpoint))) - 0.5) * 2
    return np.vectorize(sigmoid_reward)(matrix)
    #return matrix

In [3]:
matrix_path = "../embeddings/prostt5_complete_matrix_win3.pkl"
# load matrix
with open(matrix_path, "rb") as f:
    matrix_data = pickle.load(f)

In [4]:
import sys
from typing import List, Dict, Tuple

# Please do not remove package declarations because these are used by the autograder.
def OutputLCS(back, i, j):
    #print(i,j)
    if i == 0 and j == 0:
        return [], []
    if back[(i,j)] == '0':
        return [], []
    elif back[(i,j)] == 'd':
        o1,o2 = OutputLCS(back, i - 1, j)
        return o1+[str(i-1)], o2+['-']
    elif back[(i,j)] == 'r':
        o1,o2 = OutputLCS(back, i, j - 1)
        return o1+['-'], o2+[str(j-1)]
    elif back[(i,j)] == 'm':
        o1,o2 = OutputLCS(back, i - 1, j - 1)
        return o1+[str(i-1)], o2+[str(j-1)]

# Insert your local_alignment function here, along with any subroutines you need
def local_alignment(match_reward: List[list[int]], indel_penalty: int) -> Tuple[int, str, str]:
    """
    Compute the local alignment of two strings based on match reward, mismatch penalty, and indel penalty.
    """
    v, w = len(match_reward), len(match_reward[0])
    s = {}
    s[(0,0)] = 0
    back = {}
    for i in range(1, v+1):
        s[(i,0)] = max(s[(i-1,0)]-indel_penalty,0)
        if s[(i,0)] == s[(i-1,0)]-indel_penalty:
            back[(i,0)] = 'd'
        else:
            back[(i,0)] = '0'
    for j in range(1, w+1):
        s[(0,j)] = max(s[(0,j-1)]-indel_penalty,0)
        if s[(0,j)] == s[(0,j-1)]-indel_penalty:
            back[(0,j)] = 'r'
        else:
            back[(0,j)] = '0'
    for i in range(1, v+1):
        for j in range(1, w+1):
            s[(i,j)] = max(s[(i-1,j)]-indel_penalty,s[(i,j-1)]-indel_penalty,s[(i-1,j-1)]+match_reward[i-1][j-1],0)
            if s[(i,j)] == s[(i-1,j)]-indel_penalty:
                back[(i,j)] = 'd'
            elif s[(i,j)] == s[(i,j-1)]-indel_penalty:
                back[(i,j)] = 'r'
            elif s[(i,j)] == s[(i-1,j-1)]+match_reward[i-1][j-1]:
                back[(i,j)] = 'm'
            else:
                back[(i,j)] = '0'

    bestscore = max(s.values())

    for i in range(v+1):
        for j in range(w+1):
            if s[(i,j)] == bestscore:
                o1,o2 = OutputLCS(back, i, j)
    
                return bestscore,[o1[0],o1[-1]],[o2[0],o2[-1]]

In [5]:
pairname_list, comparison_id, prot1, chain1, prot2, chain2, protein1_residues, protein2_residues, alignment_score = [], [], [], [], [], [], [], [], []
for item in matrix_data.items():
    pairname, matrix = item[0], item[1]
    pairname_list.append(pairname)
    matrix = transform_similarity_matrix(matrix, midpoint=0.2, sharpness=9, scale=3)
    prot1.append(pairname.split("-")[0])
    chain1.append(pairname.split("-")[1].split("_")[0])
    prot2.append(pairname.split("-")[2])
    chain2.append(pairname.split("-")[-1].split("_")[0])
    comparison_id.append(pairname.split("-")[0]+pairname.split("-")[2])
    #print(pairname)
    score, protein1, protein2 = local_alignment(matrix, 10)
    alignment_score.append(score)
    protein1_residues.append((int(protein1[0]), int(protein1[1])+3))
    protein2_residues.append((int(protein2[0]), int(protein2[1])+3))

In [6]:
results = pd.DataFrame({
    "pairname": pairname_list,
    "comparison_id": comparison_id,
    "prot1": prot1,
    "chain1": chain1,
    "prot2": prot2,
    "chain2": chain2,
    "protein1_residues": protein1_residues,
    "protein2_residues": protein2_residues,
    "alignment_score": alignment_score
})
results.head()

,pairname,comparison_id,prot1,chain1,prot2,chain2,protein1_residues,protein2_residues,alignment_score
0,d1a05a_-A_183-d1dgsa3-A_184,d1a05a_d1dgsa3,d1a05a_,A,d1dgsa3,A,"(111, 156)","(147, 192)",67.865046
1,d1a05a_-A_185-d1j71a_-A_186,d1a05a_d1j71a_,d1a05a_,A,d1j71a_,A,"(83, 112)","(135, 164)",57.191404
2,d1a05a_-A_187-d1rblm_-M_188,d1a05a_d1rblm_,d1a05a_,A,d1rblm_,M,"(160, 177)","(18, 35)",36.232130
3,d1a2za_-A_0-d1ghha_-A_1,d1a2za_d1ghha_,d1a2za_,A,d1ghha_,A,"(189, 206)","(55, 72)",32.900426
4,d1a2za_-A_2-d1u9da_-A_3,d1a2za_d1u9da_,d1a2za_,A,d1u9da_,A,"(189, 213)","(77, 101)",53.057260


In [7]:
#results.to_csv("prostt5_regions.csv", index=False)